In [1]:
from __future__ import division
import os
from math import radians, cos, sin, asin, sqrt
import datetime
import time

In [2]:
#lista todos os estados nos dados de 2006
estados = os.listdir('2006')

In [3]:
#lista todos os arquivos de estações por estados
estacoes = {}
coordenadas_estacoes = {}
for estado in estados:
    #ja adiciona o diretorio pai para facilitar a abertura dos arquivos
    estacoes[estado] = [  '2006/'+estado+'/'+estacao for estacao in os.listdir('2006/'+estado)]
    for estacao in estacoes[estado]: #Coleta a Latitude e Longitude de cada estacao disponivel
        #print(estacao)
        with open(estacao) as estacao_arq: #Le os cabecalhos de cada arquivo (7 primeiras linhas)
            linhas = [next(estacao_arq) for x in range(7)]
        lat = linhas[4].split('UDE:;')[1].split("\n")[0].replace(',', '.') #Limpa o valor de Latitude
        lon = linhas[5].split('UDE:;')[1].split("\n")[0].replace(',', '.') #Limpa o valor de Longitude
        coordenadas_estacoes[estacao] = [float(lat), float(lon)] #Cria um dicionario de dados de estacao e lat/lon

In [4]:
#importa pandas
import pandas as pd

In [5]:
def Haversine(ponto1, ponto2):
    """
    Calcula a distancia em Kilometros entre duas estações de coleta
    """
    lat1 = ponto1[0]
    lng1 = ponto1[1]
    lat2 = ponto2[0]
    lng2 = ponto2[1]
    lat1, lng1, lat2, lng2 = map(radians, (lat1, lng1, lat2, lng2))
    lat = lat2 - lat1
    lng = lng2 - lng1
    d = sin(lat * 0.5) ** 2 + cos(lat1) * cos(lat2) * sin(lng * 0.5) ** 2
    h = 2 * 6371 * asin(sqrt(d))  #diametro em km
    return h

In [20]:
def mais_proximo(lat, lon, coordenadas_estacoes):
    """
    Encontra em um dicionario de estacoes com coordenadas a mais proxima
    """
    distancias = {}
    for estacao in coordenadas_estacoes.keys():
        distancias[Haversine( [lat,lon], coordenadas_estacoes[estacao] )] = estacao
    #print(min(distancias.keys()))
    #print(distancias[min(distancias.keys())])
    return distancias[min(distancias.keys())]

In [21]:
def coleta_coord_por_nome_municipio(nome_municipio):
    municipios = pd.read_csv('municipios.csv')
    return [ [ linha[2], linha[3] ] for linha in municipios[ municipios.nome == nome_municipio].values.tolist() ]

In [22]:
def get_chuva(arquivo_estacao):
    with open(arquivo_estacao) as arqv:
        texto = ( ''.join(arqv.readlines()[8:]).split("\n") )
        for linha in range(0, len(texto)):
            texto[linha] = texto[linha].split(";")
    arqv.close()
    df_estacao = pd.DataFrame(texto)
    df_estacao.columns = df_estacao.values[0].tolist()
    df_estacao = df_estacao.iloc[1:-1]
    return df_estacao

In [35]:
def get_total_chuva_dia(dia, dados_estacao): #YYYY-MM-DD
    chv_total = 0 
    vdf = dados_estacao[['DATA (YYYY-MM-DD)', 'HORA (UTC)', 'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)']][ dados_estacao['DATA (YYYY-MM-DD)'] == dia ]['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)']
    chv_total = sum([ j for j in [ float(i.replace(',','.')) for i in vdf.values] if j != -9999 ])
    return chv_total

In [36]:
#Mais proximo de Guarulhos por exemplo
mais_proximo(-23.413, -46.4445, coordenadas_estacoes)

'2006/SP/INMET_SE_SP_A701_SAO PAULO - MIRANTE_25-07-2006_A_31-12-2006.CSV'

In [55]:
for municipio in coleta_coord_por_nome_municipio("Divinópolis"):
    print(mais_proximo(municipio[0], municipio[1], coordenadas_estacoes))
    dados_estacao = (get_chuva(mais_proximo(municipio[0], municipio[1], coordenadas_estacoes)))
    total_de_chuva = get_total_chuva_dia('2006-12-21', dados_estacao)
    print('Total chuva', round(total_de_chuva, 2))

2006/MG/INMET_SE_MG_A524_FORMIGA_17-08-2006_A_31-12-2006.CSV
Total chuva 23.8
